In [1]:
path_model_to_eval = "experiments/object_detection/2024-09-01_11-01-16/models/model_epoch_10.pth"  # TODO CHANGE

In [2]:
import os
import sys
import argparse

print(os.getcwd())
os.chdir("../../")
os.getcwd()

sys.argv = [
    "view",
    "--config",
    "config/single_task_object_detection.yaml",
    "--model_path",
    path_model_to_eval,
]


parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
parser.add_argument(
    "--model_path", type=str, required=True, help="Path to the model pth"
)

args = parser.parse_args()

print(args.config)
print(args.model_path)

/data01/dl23vitcas/dl_project/src/single_task_object_detection
config/single_task_object_detection.yaml
experiments/object_detection/2024-09-01_11-01-16/models/model_epoch_10.pth


In [3]:
import torch
from config_experiments import config, parse_args
from utils import set_seed, set_device
from dataloader import VOC08Attr
import torchvision.transforms as transforms
from model import ObjectDetectionModel
from metrics import compute_mAP
import wandb

In [4]:
def view_mAP_for_class(mAP, data_test):
    print(f"\nmAP@0.50 (per class):")
    index = torch.arange(1, config["global"]["num_classes"] + 1)

    for i, value in zip(index, mAP["map_per_class"].numpy()):
        category = data_test.id2category.get(i.item())
        mAP_category = value.item()

        print(f"\tAP {category} : {(mAP_category):.2f}")
        wandb.config.update({f"AP {category} ": mAP_category})

    mAP50 = mAP["map_50"].item()
    print(f"\nmAP@0.50 : {mAP50:.2f}")

    wandb.config.update({"mAP@0.50": mAP50})

In [5]:
model_path = parse_args().model_path
transform_test = transforms.Compose(
    [
        transforms.Resize(size=config["transform"]["resize_values"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=config["transform"]["mean"], std=config["transform"]["std"]
        ),
    ]
)
set_seed(config["global"]["seed"])
device = set_device(config["global"]["gpu_id"])


wandb.init(
    group="object_detection",
    project="DL",
    config=config,
    save_code=True,
    mode="disabled",
)

data_test = VOC08Attr(train=False, transform=transform_test)
model = ObjectDetectionModel().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<All keys matched successfully>

In [6]:
mAP = compute_mAP(data_test, model, device)
view_mAP_for_class(mAP, data_test)

Compute mAP: 100%|██████████| 2227/2227 [07:26<00:00,  4.99it/s]



mAP@0.50 (per class):
	AP horse : 0.55
	AP person : 0.48
	AP bottle : 0.15
	AP dog : 0.38
	AP tvmonitor : 0.54
	AP car : 0.36
	AP aeroplane : 0.35
	AP bicycle : 0.47
	AP boat : 0.15
	AP chair : 0.18
	AP diningtable : 0.30
	AP pottedplant : 0.20
	AP train : 0.55
	AP cat : 0.24
	AP sofa : 0.32
	AP bird : 0.30
	AP sheep : 0.19
	AP motorbike : 0.18
	AP bus : 0.30
	AP cow : 0.34

mAP@0.50 : 0.33
